In [48]:
import json

In [49]:
import numpy as np

In [50]:
import tensorflow as tf

In [51]:
import pickle

In [52]:
import random

In [53]:
import nltk

In [54]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\av945\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [55]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\av945\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
from nltk.stem import WordNetLemmatizer

In [57]:
lemmatizer =  WordNetLemmatizer()

In [69]:
words = []
classes = [] 
documents = []
ignoreLetters = ['?','!','.',',']

In [68]:
intents = json.loads(open('intents.json').read())

In [70]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)
print("......................................................................................................................................")
print(classes)
print("......................................................................................................................................")
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
print(words)
words = sorted(set(classes))
classes = sorted(set(classes))
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['What', 'is', 'Simplilearn', '?'], 'query')]
......................................................................................................................................
['greeting', 'goodbye', 'thanks', 'query']
......................................................................................................................................
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'th

In [82]:
training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.2, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5, verbose=1, shuffle=True)
model.save('chatbot_model.h5', hist)
print('Done')

Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0778 - loss: 1.3794   
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5267 - loss: 1.1891 
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6289 - loss: 1.0150 
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4867 - loss: 1.1010 
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5689 - loss: 1.0282 
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3800 - loss: 1.1072 
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5644 - loss: 1.2056 
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6244 - loss: 0.9802 
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3956 - loss: 1.0250 
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6022 - loss: 1.0223 
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6378 - loss: 0.8787 
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4222 - l

Done


In [1]:
from sklearn.model_selection import train_test_split 

In [2]:
messages = ["Hey, let's not got the class today!",
           "Call your Friend!",
           "Want to go for the walk tomorrow?"]

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [22]:
msg = vect.fit_transform(messages)
vect.get_feature_names_out()

array(['call', 'class', 'for', 'friend', 'go', 'got', 'hey', 'let', 'not',
       'the', 'to', 'today', 'tomorrow', 'walk', 'want', 'your'],
      dtype=object)

In [23]:
print(msg.toarray())

[[0.         0.38988801 0.         0.         0.         0.38988801
  0.38988801 0.38988801 0.38988801 0.29651988 0.         0.38988801
  0.         0.         0.         0.        ]
 [0.57735027 0.         0.         0.57735027 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.57735027]
 [0.         0.         0.38988801 0.         0.38988801 0.
  0.         0.         0.         0.29651988 0.38988801 0.
  0.38988801 0.38988801 0.38988801 0.        ]]


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
dt = vect.fit_transform(messages)

In [17]:
print(dt)

  (0, 11)	0.3898880096169543
  (0, 1)	0.3898880096169543
  (0, 9)	0.29651988085384556
  (0, 5)	0.3898880096169543
  (0, 8)	0.3898880096169543
  (0, 7)	0.3898880096169543
  (0, 6)	0.3898880096169543
  (1, 3)	0.5773502691896257
  (1, 15)	0.5773502691896257
  (1, 0)	0.5773502691896257
  (2, 12)	0.3898880096169543
  (2, 13)	0.3898880096169543
  (2, 2)	0.3898880096169543
  (2, 4)	0.3898880096169543
  (2, 10)	0.3898880096169543
  (2, 14)	0.3898880096169543
  (2, 9)	0.29651988085384556
